### 我在一个函数中打开了一个图像, 然后将图像的矩阵返回到调用函数中, 
### 调用函数需要继续操作矩阵并生成新矩阵, 本经聚合矩阵为新矩阵的结果只得到了一个一维的包含就矩阵地址的一维数组
### 研究下python的参数传递用法, 然后解决这个问题.


In [22]:
import numpy as np
import cv2
import glob
import os

CROP_SIZE=448
img_dir = r"F:\workspace\empty_scan_test\testSet5.0_lit\03_04L01"

In [23]:
def get_image_list(root):
    extension = '.png'
    pattern = os.path.join(root, f"*{extension}")
    imglist = glob.glob(pattern)
    imglist.sort()
    #print(imglist)
    return imglist

In [61]:
def crop_img_and_remove_black_edge(img, true_width_left: int,
                                   true_width_right: int):
    r, c = img.shape[:2]
    true_width = true_width_right - true_width_left + 1
    if true_width < 1:
        raise ValueError("true width wrong!")
    row_start = 0
    if true_width > r:
        raise ValueError(f"true width can not be cuted r{r}c{c}t{true_width}")
    else:
        col_start = true_width_left
        size = true_width
    xshift = row_start
    yshift = 0
    return img[row_start:row_start + size, col_start:col_start +
               size], xshift, yshift

def get_image_actual_width_index_range_mem(imagebuff=None):
    """
        根据已经读入内存的图片的矩阵获取图像有效宽度, 输入参数类型为numpy.ndarray
        返回tuple(left, right) 代表有效列的索引
    """
    if not imagebuff.any() or not isinstance(imagebuff, np.ndarray):
        raise ValueError("param is not np.ndarray")
    if len(imagebuff.shape) != 2 and len(imagebuff.shape) != 3:
        raise ValueError(
            "image must in IMREAD_GRAYSCALE or IMREAD_COLOR(height, row, gray)!"
        )

    gray_img = imagebuff.sum(axis=2) if len(imagebuff) == 3 else imagebuff

    gray_img_t = gray_img.transpose()

    left = 0
    right = gray_img_t.shape[0] - 1
    for col in range(gray_img_t.shape[0]):
        column = gray_img_t[col]
        column.sort()
        top20 = column[-20:]
        if top20.mean() > 10:
            left = col
            break
    for col in range(gray_img_t.shape[0] - 1, -1, -1):
        column = gray_img_t[col]
        column.sort()
        top20 = column[-20:]
        if top20.mean() > 10:
            right = col
            break
    return (left, right)


def get_image_actual_width_index_range(imagefile=None):
    """
        获取图片的实际有效宽度的索引范围
    """
    if not imagefile:
        raise ValueError("imagefile is None")
    if not os.path.isfile:
        msg = f"imagefile {imagefile} not exist"
        raise ValueError(msg)
    img = cv2.imread(imagefile, cv2.IMREAD_COLOR)
    if not img.size:
        raise ValueError("imagefile read None")
        return None
    else:
        return get_image_actual_width_index_range_mem(img)

def correct_image_size(img_array, zoom=False):
    """
        crop+resize. 返回处理好的图片并返回X轴偏移以及缩放比例. 
    """
    #print(f"correct_image_size: array shape is {img_array.shape}, type({type(img_array)})")
    if not img_array.any() or not isinstance(img_array, np.ndarray):
        raise ValueError("param is not np.ndarray")
    if len(img_array.shape) != 2 and len(img_array.shape) != 3:
        raise ValueError(
            "image must in IMREAD_GRAYSCALE or IMREAD_COLOR(height, row, gray)!"
        )
    true_width = get_image_actual_width_index_range_mem(img_array.copy())
    img_croped, xshift, _ = crop_img_and_remove_black_edge(
        img_array, true_width[0], true_width[1])
    if zoom:
        img_array_new = cv2.resize(img_croped,
                                   dsize=(CROP_SIZE, CROP_SIZE),
                                   interpolation=cv2.INTER_AREA)
    else:
        img_array_new = img_croped
    ratio = img_array_new.shape[0] / img_croped.shape[0]
    return img_array_new, xshift, ratio

def read_image_and_correct_it(imagefile=None, zoom=False):
    """
    读取图片, 如果图片不是448*448的那么进行crop, resize
    注意: 此函数调用后需要配套修改json的坐标
    """
    #print(f"read_image_and_correct_it: file is {imagefile}")
    if not imagefile:
        raise ValueError("imagefile is None")
    if not os.path.isfile:
        msg = f"imagefile {imagefile} not exist"
        raise ValueError(msg)
    img = cv2.imread(imagefile, cv2.IMREAD_GRAYSCALE)
    print("read_image_and_correct_it: img id", id(img))
    if not img.size:
        raise ValueError("imagefile read None")
    #print(f"read_image_and_correct_it: array shape is {img.shape}, type({type(img)})")
    xshift = 0
    ratio = 1
    if img.shape[0] == CROP_SIZE:
        new_img = img
    else:
        new_img, xshift, ratio = correct_image_size(img, zoom)

    return (new_img, xshift, ratio)

In [37]:
def read_image_and_correct_it2(imagefile=None, imgs=None, zoom=False):
    """
    读取图片, 如果图片不是448*448的那么进行crop, resize
    注意: 此函数调用后需要配套修改json的坐标
    """
    #print(f"read_image_and_correct_it: file is {imagefile}")
    if not imagefile:
        raise ValueError("imagefile is None")
    if not os.path.isfile:
        msg = f"imagefile {imagefile} not exist"
        raise ValueError(msg)
    imgs[len(imgs)-1] = cv2.imread(imagefile, cv2.IMREAD_GRAYSCALE)
    img = imgs[len(imgs)-1]
    print("read_image_and_correct_it: img id", id(img))
    if not img.size:
        raise ValueError("imagefile read None")
    #print(f"read_image_and_correct_it: array shape is {img.shape}, type({type(img)})")
    xshift = 0
    ratio = 1
    if img.shape[0] == CROP_SIZE:
        new_img = img
    else:
        new_img, xshift, ratio = correct_image_size(img)

    return (new_img, xshift, ratio)

In [33]:
imgfiles = get_image_list(img_dir)

imgs = []

for imgfile in imgfiles:
    img_datas = read_image_and_correct_it(imgfile)
    img = img_datas[0]
    print("img id", id(img))
    xshift = img_datas[1]
    ratio = img_datas[2]
    imgs.append(img)
    

    

read_image_and_correct_it: img id 2103163142832
img id 2103163142832
read_image_and_correct_it: img id 2103163147488
img id 2103163147488
read_image_and_correct_it: img id 2103163149008
img id 2103163149008
read_image_and_correct_it: img id 2103163148928
img id 2103163147408
read_image_and_correct_it: img id 2103163126208
img id 2103163149648
read_image_and_correct_it: img id 2103163149568
img id 2103163149328
read_image_and_correct_it: img id 2103163182656
img id 2103163182016


In [50]:

def resize(img: np.ndarray, size: int=None, interpolation: int=cv2.INTER_AREA) -> np.ndarray:
    """
    Returns a square resized image using `cv2.resize`

    Args:
        img (ndarray): Single image array.
        min_size (int): Optional output size (otherwise uses minimum or rows and cols)
        interpolation (int): cv2 interpolation method (default cv2.INTER_AREA)

    Returns:
        ndarray: resized image array
    """
    r, c, nch = img.shape
    if size is None:
        size = min(r, c)
    #print(f"transform_resize: before shape{img.shape}")
    new_img = cv2.resize(img, (size, size), interpolation=interpolation)
    #print(f"transform_resize: after1 shape{img.shape}, new{new_img.shape}")
    new_img2 = new_img.reshape(size, size, nch)
    #print(f"transform_resize: after2 shape{new_img.shape}, new{new_img2.shape}")
    #return cv2.resize(img, (size, size), interpolation=interpolation).reshape(size, size, nch)
    return new_img

In [36]:
import copy

img_list = [img[..., None] for img in imgs]
print(">>before process img_list", len(img_list), 'img', img_list[0].shape, id(img_list[0]))
img_array = np.array(copy.deepcopy(img_list))
print(">>after process img_array.shape", img_array.shape, type(img_array))

>>before process img_list 7 img (448, 448, 1) 2103163126848
>>after process img_array.shape (7,) <class 'numpy.ndarray'>


In [38]:
imgfiles = get_image_list(img_dir)

imgs = []

for imgfile in imgfiles:
    imgs.append(None)
    img_datas = read_image_and_correct_it2(imgfile, imgs)
    #img = img_datas[0]
    print("img id", id(img))
    xshift = img_datas[1]
    ratio = img_datas[2]
    imgs.append(img)

read_image_and_correct_it: img id 2103141058960
img id 2103141058960
read_image_and_correct_it: img id 2103162748160
img id 2103162748160
read_image_and_correct_it: img id 2103163147408
img id 2103163147408
read_image_and_correct_it: img id 2103163149648
img id 2103163149328
read_image_and_correct_it: img id 2103163308032
img id 2103163308512
read_image_and_correct_it: img id 2103163145072
img id 2103163308672
read_image_and_correct_it: img id 2103163308592
img id 2103163308832


In [54]:
imgs_a = np.array(imgs)
print("a", imgs_a.shape)
img_list = [img[..., None].reshape(*(img.shape), 1) for img in imgs]
print(">>before process img_list", len(img_list), 'img', img_list[0].shape, id(img_list[0]))
img_array = np.array(img_list)
print(">>after process img_array.shape", img_array.shape, type(img_array))
print(id(img_list))
for i in range(len(img_list)):
    print(id(img_list[i]))

a (14,)
>>before process img_list 14 img (448, 448, 1) 2103163280880
>>after process img_array.shape (14,) <class 'numpy.ndarray'>
2103163209864
2103163280880
2103163311568
2103163311648
2103163311728
2103163311808
2103163311888
2103163311968
2103163312048
2103163312128
2103163312208
2103163312288
2103163312368
2103163312448
2103163312528


In [45]:
imgs_a = np.array(imgs)
print("a", imgs_a.shape)
img_list = [resize(img[..., None], 448) for img in imgs]
print(">>before process img_list", len(img_list), 'img', img_list[0].shape, id(img_list[0]))
img_array = np.array(img_list)
print(">>after process img_array.shape", img_array.shape, type(img_array))

a (14,)
transform_resize: before shape(448, 448, 1)
transform_resize: after1 shape(448, 448, 1), new(448, 448)
transform_resize: after2 shape(448, 448), new(448, 448, 1)
transform_resize: before shape(448, 448, 1)
transform_resize: after1 shape(448, 448, 1), new(448, 448)
transform_resize: after2 shape(448, 448), new(448, 448, 1)
transform_resize: before shape(448, 448, 1)
transform_resize: after1 shape(448, 448, 1), new(448, 448)
transform_resize: after2 shape(448, 448), new(448, 448, 1)
transform_resize: before shape(448, 448, 1)
transform_resize: after1 shape(448, 448, 1), new(448, 448)
transform_resize: after2 shape(448, 448), new(448, 448, 1)
transform_resize: before shape(448, 448, 1)
transform_resize: after1 shape(448, 448, 1), new(448, 448)
transform_resize: after2 shape(448, 448), new(448, 448, 1)
transform_resize: before shape(448, 448, 1)
transform_resize: after1 shape(448, 448, 1), new(448, 448)
transform_resize: after2 shape(448, 448), new(448, 448, 1)
transform_resize: be

In [51]:
imgs_a = np.array(imgs)
print("a", imgs_a.shape)
img_list = [resize(img[..., None], 448) for img in imgs]
print(">>before process img_list", len(img_list), 'img', img_list[0].shape, id(img_list[0]))
img_array = np.array(img_list)
print(">>after process img_array.shape", img_array.shape, type(img_array))

a (14,)
>>before process img_list 14 img (448, 448) 2103163280960
>>after process img_array.shape (14, 448, 448) <class 'numpy.ndarray'>


In [53]:
print(id(img_list))
for i in range(len(img_list)):
    print(id(img_list[i]))

2103163150984
2103163280960
2103163310128
2103163310208
2103163310288
2103163126208
2103163310368
2103163310448
2103163310528
2103163310608
2103163310688
2103163310768
2103163310848
2103163310928
2103163311008


In [62]:
imgfiles = get_image_list(img_dir)

imgs = []

for imgfile in imgfiles:
    img_datas = read_image_and_correct_it(imgfile, True)
    img = img_datas[0]
    print("img id", id(img))
    xshift = img_datas[1]
    ratio = img_datas[2]
    imgs.append(img)
img_list = [img[..., None] for img in imgs]
for i in img_list:
    print(i.shape)
print(">>before process img_list", len(img_list), 'img', img_list[0].shape, id(img_list[0]))
img_array = np.stack(imgs, axis=0)
print(">>after process img_array.shape", img_array.shape, type(img_array))
print(id(img_list))


read_image_and_correct_it: img id 2103163451712
img id 2103163451712
read_image_and_correct_it: img id 2103163452032
img id 2103163452032
read_image_and_correct_it: img id 2103163328352
img id 2103163328352
read_image_and_correct_it: img id 2103163328272
img id 2103163326752
read_image_and_correct_it: img id 2103163452432
img id 2103163329312
read_image_and_correct_it: img id 2103163452432
img id 2103163327072
read_image_and_correct_it: img id 2103163452432
img id 2103163326512
(448, 448, 1)
(448, 448, 1)
(448, 448, 1)
(448, 448, 1)
(448, 448, 1)
(448, 448, 1)
(448, 448, 1)
>>before process img_list 7 img (448, 448, 1) 2103163452432
>>after process img_array.shape (7, 448, 448) <class 'numpy.ndarray'>
2103163282376
